---

# AI 技术、概念和算法简介

---

1. 神经网络、损失函数与深度学习技术

2. 环境引擎、奖励函数与强化学习技术

3. AI & CFD


---

## 神经网络、损失函数与深度学习技术

---

+ 1.0. 通过数据统计做预测

+ 1.1. 通过线性拟合函数来更快地预测结果

+ 1.2. 通过监督方法来更快地更新函数参数

+ 1.3. 非线性函数的拟合

+ 1.4. 神经网络的前向计算和反向参数更新

+ 1.5. RNN 和 LSTM 模型


In [3]:
# 通过历史数据来预测未曾见到过的数据

# 通过数据统计来预测结果

# K Nearest Neighbors 方案

import numpy as np

def knn(query_x, y, top_n=3):
    sorted_notes = sorted(y.items(), key=lambda xy: (xy[0] - query_x)**2)
    similary_notes = sorted_notes[:top_n]
    similary_ys = [y for _, y in similary_notes]
    return np.mean(similary_ys)

### 通过线性拟合函数来更快地预测结果:

**线性拟合**

$$y = f(x) = k*x + b$$

**损失函数**

$$loss(y, \hat{y}) = \frac{1}{N} {\sum_{i \in N}(y_i - \hat{y_i})^2} = \frac{1}{N} {\sum_{i \in N}(k*x_i + b - \hat{y_i})^2}$$ 

In [4]:
# 通过随机方法搜索参数，通过损失函数判断、保留当前参数组合

import random

def loss(y_hat, y):
    return np.mean((y_hat - y)**2)

def random_approach(x, y, max_steps=1000):
    best_k, best_b = None, None    
    min_loss = float('inf')
    min_v, max_v = -100, 100
    for _ in range(max_steps):
        k, b = random.randrange(min_v, max_v), random.randrange(min_v, max_v)
        y_hats = [k*rm_i+b for rm_i in x]
        curr_loss = loss(y_hats, y)
        if curr_loss < min_loss:
            min_loss = curr_loss
            best_k = k
            best_b = b
    return best_k, best_b

### 通过监督方法来更快地更新函数参数:

**损失函数梯度**

$$\frac{\partial{Loss(k, b)}}{\partial{k}} = \frac{2}{N} {\sum_{i \in N}(k*x_i+b - \hat{y_i})*x_i} \space
,\space \frac{\partial{Loss(k, b)}}{\partial{b}} = \frac{2}{N} {\sum_{i \in N}(k*x_i+b - \hat{y_i})}$$   

**误差反向传播**

$$k_{n+1} = k_{n} + (-1)*\frac{\partial{Loss(k, b)}}{\partial{k}}*\mathrm{d}k \space
,\space
b_{n+1} = b_{n} + (-1)*\frac{\partial{Loss(k, b)}}{\partial{b}}*\mathrm{d}b $$

*沿着逆梯度方向损失函数下降得最快！*

In [5]:
# 通过蒙特卡洛方法搜索参数，通过损失函数判断、保留当前参数组合

def partial_k(k, b, x, y):
    return 2*np.mean((k*x + b - y)*x)

def partial_b(k, b, x, y):
    return 2*np.mean(k*x + b - y)

def montecarlo_approach(query_x, y, learn_rate, steps):
    k, b = random.random(), random.random()
    min_loss = float('inf')
    for _ in range(steps):
        k = k + partial_k(k, b, query_x, y)*(-1)*learn_rate
        b = b + partial_b(k, b, query_x, y)*(-1)*learn_rate
        y_hats = [k*x + b for x in query_x] 
        curr_loss = loss(y_hats, y)

        if curr_loss < min_loss:
            min_loss = curr_loss
    return (k, b, min_loss)

### 非线性函数的拟合：

1. 使用基本的、简单的 **元函数** 就可以拟合非常多的复杂的函数；
2. 组合线性函数和非线性函数就可以拟合各种曲线。  

**sigmoid 函数**

$$sigmoid(x) = \frac{1}{1+e^{x}}$$

In [ ]:
from matplotlib import pyplot as plt

def sigmoid(x):
    return 1 / (1 + np.exp(-x))

# sigmoid 函数图像。
x = np.linspace(-10, 10)
plt.title("sigmod")
plt.plot(x, sigmoid(x))

In [ ]:
def random_linear(x):
    k, b = random.normalvariate(0, 1), random.normalvariate(0, 1)
    return k*x + b

# 通过一个线性函数random_linear和一个非线性函数 sigmoid 拟合各种曲线。
x = np.linspace(-10, 10)
plt.title("linear+sigmoid")
for i in range(10):
    plt.plot(x, random_linear(sigmoid(random_linear(x)))) 

### 现实的数据关系可能是线性的、非线性的、高度非线性的，复杂的映射关系，需要复杂的嵌套表达方法:

**复合函数**  

$$linear_1 = k_1*x + b_1 \space \Rightarrow \space linear_2 = k_2*sigmoid(linear_1) + b_2$$

$$ \Rightarrow \quad Loss(k_1, k_2, b_1, b_2) = \frac{1}{N} {\sum_{i \in N}(linear_2 - \hat{y_i})^2} $$


**链式求导**  

$$\begin{gathered}
\frac{\partial{Loss}}{\partial{k_1}} = \frac{\partial{Loss}}{\partial{linear_2}} *\frac{\partial{linear_2}}{\partial{sigmoid}} *\frac{\partial{sigmoid}}{\partial{linear_1}} *\frac{\partial{linear_1}}{\partial{k_1}} \\
\\
\frac{\partial{Loss}}{\partial{b_1}} = \frac{\partial{Loss}}{\partial{linear_2}} *\frac{\partial{linear_2}}{\partial{sigmoid}} *\frac{\partial{sigmoid}}{\partial{linear_1}} *\frac{\partial{linear_1}}{\partial{b_1}} \\
\\
\frac{\partial{Loss}}{\partial{k_2}}、\frac{\partial{Loss}}{\partial{b_2}} ...\\
\end{gathered}$$

**梯度下降**

$$\begin{gathered}
k_{1_{n+1}} = k_{1_{n}} + (-1)*\frac{\partial{Loss}}{\partial{k_1}}*\mathrm{d}k  \\
b_{1_{n+1}} = b_{1_{n}} + (-1)*\frac{\partial{Loss}}{\partial{b_1}}*\mathrm{d}b  \\
k_{2_{n+1}}、b_{2_{n+1}} ...
\end{gathered}$$

这就是深度学习技术最初最基本的技术思路：通过神经网络层之间不断嵌套的函数组合来表征任意复杂度的映射关系，通过神经网络层内部神经元来表达各层上的函数参数，通过损失函数的梯度来不断更新深度神经网络。

---

### RNN 和 LSTM 模型

---

长短时模型（LSTM）是循环神经网络（RNN）的一种特殊类型。与传统的神经网络不同，RNN 具有循环结构，允许信息在网络中持续流动。这使得 RNN 在理论上可以记住长期的信息。然而，实际上，标准的 RNN 在尝试捕获长期依赖关系时遇到了困难。

+ 1.5.1. RNN 模型的原理和局限性
+ 1.5.2. LSTM 模型的结构和改进


**参考资料：**

+ [机器学习杂货店 - 长短期神经网络(LSTM)原理及实现](https://mp.weixin.qq.com/s?search_click_id=10126261854878958402-1706075590040-0111285852&__biz=MzkwNjMwOTYwOA==&mid=2247491963&idx=1&sn=30aeb87b11354bf82f4fb91016f82a0b&chksm=c0e8dbfcf79f52eaa68b7d3f4fb20bef492165f056dc7adc83cc972eeea17968ade6949f50de&key=23289373de30ed987097a18ad14296064b363b66d3f49002753f373dbe2177ec3660d5aeabbb9fd0686d286e295eef9c335412355869ed700207a77a8bcaab4f544e76abb38838bebd4ab7198839bf63b465be520a93b7e62c4e8058c7f5e32fe632a74153032b9df346a70c0bca904bcd8936625be0ff7d364f08bd8a84e74c&ascene=0&uin=MTM1MjY1OTIxOQ%3D%3D&devicetype=Windows+10+x64&version=63090819&lang=zh_CN&countrycode=CN&exportkey=n_ChQIAhIQ7zoOWdlqwtGe8bp%2FK0yRWxLgAQIE97dBBAEAAAAAADC7C%2BZTMHoAAAAOpnltbLcz9gKNyK89dVj0bcz7rYnwY7YJdNBTeLwOQSMd5XlVUONwHzruNSMsV2akTT%2B3GdmHDI7AALQH6a1UGzlvfYQsrwF1J8f6vz0pbriVNFMDpuw8zRQitghM1uEorBu2K2J60OCW2KR0jN51oaukD%2BkNDwzX4D40WF9qijxzDZx3OSHzKcWiOyblJIkfkMm9P5dj18ODcNvylsfILiKKJ5PeCqMzXP7r7z0jayX2RRFjEIRADGavKaH9I41u7clnwGjcl5mW&acctmode=0&pass_ticket=rYrw3FUKfboINOlh9sJPMaon0MgtUGhzxsOmPVxTe4ufnYTOrIJyGBMIMfyXZWBMyIRzgZVYJ%2FB95cHkyeXtWQ%3D%3D&wx_header=1&fasttmpl_type=0&fasttmpl_fullversion=7044993-zh_CN-zip&fasttmpl_flag=0)

---

1.5.1. RNN 模型的原理和局限性

---

要更好地理解 LSTM 模型应用在时间序列预报场景下的优势，我们最好能先对它的前身 RNN 模型有所了解。

RNN是一种特殊的神经网路结构，其本身是包含循环的网络，允许信息在神经元之间传递。

![](./assets/rnn0.png)

图示是一个 RNN 结构示意图，图中的 $A$ 表示神经网络模型，$X_t$ 表示模型的输入信号，$h_t$ 表示模型的输出信号。与普通的神经网络结构的最大不同在于，RNN 模型中允许 $A$ 将信息传递给 $A$ ，神经网络可以将自己的输出作为输入！

输入信号是一个时间序列，跟时间 $t$ 有关。也就是说，在 $t$ 时刻，输入信号 $X_t$ 作为神经网络 $A$ 的输入，$A$ 的输出分流为两部分，一部分输出给 $h_t$，另一部分作为一个隐藏的信号流被输入到 $A$ 中。在下一次时刻输入信号 $X_{t+1}$ 时，这部分隐藏的信号流也作为输入信号输入到了 $A$ 中。此时，神经网络 $A$ 就同时接收了 $X_t$ 时刻和 $X_{t+1}$ 时刻的信号输入，此时的输出信号又将被传递到下一时刻的 $A$ 中。如果我们把上面那个图根据时间 $t$ 展开来看，就是:

![](./assets/rnn1.png)

$t = 0$ 时刻的信息输出给 $t = 1$ 时刻的模型 $A$，$t = 1$ 时刻的信息输出给 $t = 2$ 时刻的模型 $A$ ，...。这样链式的结构揭示了 RNN 本质上是与序列相关的，是对于时间序列数据最自然的神经网络架构。理论上，RNN 可以保留以前任意时刻的信息。但在实际使用的时候，用得最多的一种 RNN 结构是 LSTM，为什么呢? 

**RNN的局限性**

RNN 利用了神经网络的 “内部循环” 来保留时间序列的上下文信息，可以使用过去的信号数据来推测对当前信号的理解，这非常重要，并且理论上 RNN 可以保留过去任意时刻的信息。但实际使用 RNN 时往往遇到问题，比如在这个例子里：

```
假如我们构造了一个语言模型，可以通过当前这一句话的意思来预测下一个词语。现在有这样一句话：

“我是一个中国人，出生在普通家庭，我最常说汉语，也喜欢写汉字。我喜欢妈妈做的菜”。

我们的语言模型在预测“我最常说汉语”的“汉语”这个词时，它要预测“我最常说”这后面可能跟的是一个语言，可能是英语，也可能是汉语，那么它需要用到第一句话的“我是中国人”这段话的意思来推测我最常说汉语，而不是英语、法语等。

而在预测“我喜欢妈妈做的菜”的最后的词“菜”时并不需要“我是中国人”这个信息以及其他的信息，它跟我是不是一个中国人没有必然的关系。
```

这个例子告诉我们，想要精确地处理时间序列，有时候我们只需要用到最近的时刻的信息。例如预测“我喜欢妈妈做的菜”最后这个词“菜”，此时信息传递是这样的：

![](./assets/rnn2.png)

“菜”这个词与“我”、“喜欢”、“妈妈”、“做”、“的”这几个词关联性比较大，距离也比较近，所以可以直接利用这几个词进行最后那个词语的推测。

而有时候我们又需要用到很早以前时刻的信息，例如预测“我最常说汉语”最后的这个词“汉语”。此时信息传递是这样的：

![](./assets/rnn3.png)

此时，我们要预测“汉语”这个词，仅仅依靠“我”、“最”、“常”、“说”这几个词还不能得出我说的是汉语，必须要追溯到更早的句子“我是一个中国人”，由“中国人”这个词语来推测我最常说的是汉语。因此，这种情况下，我们想要推测“汉语”这个词的时候就比前面那个预测“菜”这个词所用到的信息就处于更早的时刻。

RNN 虽然在理论上可以保留所有历史时刻的信息，但在实际使用时，信息的传递往往会因为时间间隔太长而逐渐衰减，传递一段时刻以后其信息的作用效果就大大降低了。因此，普通 RNN 对于信息的长期依赖问题没有很好的处理办法。

---

1.5.2. LSTM 模型的结构和改进

---

长短期记忆（Long Short Term Memory，LSTM）网络是一种特殊的 RNN 模型，其特殊的结构设计使得它可以避免长期依赖问题，记住很早时刻的信息是 LSTM 的默认行为，而不需要专门为此付出很大代价。

普通的 RNN 模型中，其重复神经网络模块的链式模型如下图所示，这个重复的模块只有一个非常简单的结构，一个单一的神经网络层（例如 tanh 层），这样就会导致信息的处理能力比较低。

![](./assets/rnn4.png)

而 LSTM 在此基础上将这个结构改进了，不再是单一的神经网络层，而是4个，并且以一种特殊的方式进行交互。

![](./assets/rnn5.png)

粗看起来，这个结构有点复杂，我们先来认识一些基本的模块表示方法。图中的模块分为以下几种：

![](./assets/rnn6.png)

LSTM 的关键是细胞状态（cell state），表示为 $C_t$，用来保存当前 LSTM 的状态信息并传递到下一时刻的 LSTM 中，也就是 RNN 中的 “自循环“ 的过程。当前的 LSTM 接收来自上一个时刻的细胞状态 $C_{t-1}$，并与当前 LSTM 接收的信号输入 $x_t$ 共同作用产生当前 LSTM 的细胞状态 $C_t$。

![](./assets/rnn7.png)

LSTM 中采用专门设计的 "门" 来引入或者去除细胞状态 $C_t$ 中的信息。门是一种让信息选择性通过的方法。有的门跟信号处理中的滤波器类似，允许信号部分通过或者通过时被门加工了；有的门跟数字电路中的逻辑门类似，允许信号通过或者不通过。这里所釆用的门包含一个 sigmoid 神经网络层和一个按位的乘法操作，如下图所示:

![](./assets/rnn8.png)

sigmoid 神经网络层可以将输入信号转换为 0 到 1 之间的数值，用来描述有多少量的输入信号可以通过。0 表示 “不允许任何量通过"，1 表示 "允许所有量通过"。

LSTM 主要包括三个不同的门结构: 遗忘门、记忆门和输出门。这三个门用来控制 LSTM 的信息保留和传递，最终反映到细胞状态 $C_t$ 和输出信号 $h_t$。如下图所示:

![](./assets/rnn9.png)

图中标示了 LSTM 中各个门的构成情况和相互之间的关系，其中:

+ 遗忘门由一个sigmoid神经网络层和一个按位乘操作构成；
+ 记忆门由输入门 (input gate) 与tanh神经网络层和一个按位乘操作构成；
+ 输出门 (output gate) 与 tanh 函数以及按位乘操作共同作用将细胞状态和输入信号传递到输出端。

**遗忘门**

顾名思义，遗忘门的作用就是用来 "忘记" 信息的。在 LSTM 的使用过程中，有一些信息不是必要的，因此遗忘门的作用就是用来选择这些信息并 "忘记" 它们。遗忘门决定了细胞状态 $C_t$ 中的哪些信息将被遗忘。

![](./assets/rnn10.png)

![](./assets/rnn14.gif)

左边高亮的结构就是遗忘门，包含一个 sigmoid 神经网络层（神经网络参数为 $W_f, b_f$），接收 $t$ 时刻的输入信号 $x_t$ 和 $t-1$ 时刻 LSTM 的上一个输出信号 $h_{t-1}$，这两个信号进行垪接以后共同输入到 sigmoid 神经网络层中，然后输出信号 $f_t$。$f_t$ 是一个 0 到1之间的数值，并与 $C_{t-1}$ 相乘来决定 $C_{t-1}$ 中的哪些信息将被保留，哪些信息将被舍弃。 
注意，此时 $f_t$ 是一个与 $C_t$ 维数相同的向量。同样要注意的是，不要混淆这里的 sigmoid 神经网络层和 sigmoid 激活函数，两者不是一个东西。

**记忆门**

记忆门的作用与遗忘门相反，它将决定新输入的信息 $x_t$ 和 $h_{t-1}$ 中哪些信息将被保留。

![](./assets/rnn11.png)

![](./assets/rnn13.gif)

如图所示，记忆门包含 2 个部分。第一个是包含 sigmoid 神经网络层（输入门，神经网络网络参数为 $W_i, b_i$）和一个 tanh 神经网络层（神经网络参数为 $W_c, b_c$）。

+ sigmoid 神经网络层的作用很明显，跟遗忘门一样，它接收 $x_t$ 和 $h_{t-1}$ 作为输入，然后输出一个 0 到 1 之间的数值 $i_t$ 来决定哪些信息需要被更新；
+ Tanh神经网络层的作用是将输入的 $x_t$ 和 $h_{t-1}$ 整合，然后通过一个 tanh 神经网络层来创建一个新的状态候选向量 $\hat C_t$, 值范围在 -1 到 1 之间。记忆门的输出由上述两个神经网络层的输出决定，$i_t$ 与 $\hat C_t$ 相乘来选择哪些信息将被新加入到 $t$ 时刻的细胞状态 $C_t$ 中。

**更新细胞状态**

有了遗忘门和记忆门，我们就可以更新细胞状态 $C_t$ 了。

![](./assets/rnn12.png)

这里将遗忘门的输出 $f_t$ 与上一时刻的细胞状态 $C_{t-1}$ 相乘来选择遗忘和保留一些信息，将记忆门的输出与从遗忘门选择后的信息加和得到新的细胞状态 $C_t$。这就表示 $t$ 时刻的细胞状态 $C_t$ 已经包含了此时需要丟弃的 $t-1$ 时刻传递的信息和 $t$ 时刻从输入信号获取的需要新加入的信息 $i_t \cdot \hat C_t$ 。 $C_t$ 将继续传递到 $t+1$ 时刻的 LSTM网络中，作为新的细胞状态传递下去。

**输出门**

前面已经讲了LSTM如何来更新细胞状态 $C_t$，那么在 $t$ 时刻我们输入信号 $x_t$ 以后，对应的输出信号该如何计算呢?

![](./assets/rnn15.gif)

输出门整合的过程如上图所示，$x_t$ 和 $h_{t-1}$ 经过一个 sigmoid 神经网络层（神经网络参数为 $W_o, b_o$）输出一个 0 到 1 之间的数值 $o_t$。经过前面遗忘门与记忆门选择后的细胞状态 $C_t$ 再经过一个 tanh 函数（注意：这里不是神经网络层）得到一个在 -1 到 1 之间的数值，并与 $o_t$ 相乘得到输出信号 $h_t$，同时 $h_t$ 也作为下一个时刻的输入信号传递到下一阶段。


---

## 环境引擎、奖励函数与强化学习技术

---

在强化学习 (Reinforcement Learning, RL) 中，智能体（算法，Agent）通过与环境（Environment）的交互来学习动作策略，从而最大化自己在环境中所获得的奖励。监督学习关注的是“预测”，强化学习关注的是“决策”。
通过不断与环境交互、观察奖励信号和更新策略参数，深度强化学习算法能够通过试错学习的方式发现最优策略。在训练的过程中，神经网络会逐渐收敛，从而使智能体能够在实际应用中做出最佳决策。 

+ 2.1. 智能体和环境
+ 2.2. 马尔可夫决策过程

---

### 智能体和环境

---

强化学习也有训练过程，“智能体”需要不断观察“环境”状态、依据相应策略执行动作，观察执行动作后的反馈，积累经验形成一个模型。与有监督学习不同的是，这里每个动作一般没有直接标定的标签值作为监督信号，系统只给算法执行的动作一个反馈。

强化学习算法要通过样本学习得到一个映射函数，称为策略函数，其输入是当前时刻环境信息，输出是要执行的动作：

$a = \pi (s)$

类似于有监督学习中需要定义损失函数来评价预测函数的优劣，在强化学习中也需要对策略函数的优劣进行评价。状态价值函数，它是在某个状态 $s$ 下，按照策略π执行动作，累计回报的数学期望；衡量的是按照某一策略执行之后的累计回报。

$V_{\pi}(s) = \sum \limits_{s'} p_{\pi (s)}(s, s') (r_{\pi (s)}(s, s') + \gamma V_{\pi}(s'))$

这是一个递归的定义，函数的自变量是状态与策略函数，将它们映射成一个实数，每个状态的价值函数依赖于从该状态执行动作后能到达的后续状态的价值函数。在状态 $s$ 时执行动作 $\pi (s)$，下一时刻的状态 $s'$ 是不确定的，进入每个状态的概率为 $p_{\pi (s)}(s, s')$,当前获得的回报是 $r_{\pi (s)}(s, s')$，因此需要对下一时刻所有状态计算数学期望即概率意义上的均值，而总的回报包括当前的回报，以及后续时刻的回报值之和即 $V_{\pi}(s')$。在这里 $r_{\pi (s)}(s, s')$ 表示当前时刻获得的回报，$\gamma$ 表示期望折扣系数。

类似的可以定义动作价值函数。它是智能体按照策略 $\pi$ 执行，在状态 $s$ 时执行具体的动作 $a$ 后的预期回报：

$Q_{\pi}(s, a) = \sum \limits_{s'} p_{a}(s, s') (r_{a}(s, s') + \gamma V_{\pi}(s'))$

动作价值函数除了指定初始状态 $s$与策略 $\pi$ 之外，还指定了在当前的状态 $s$ 时执行的动作 $a$。这个函数衡量的是按照某一策略，在某一状态时执行各种动作的价值。这个值等于在当前状态 $s$ 下执行一个动作后的立即回报 $r_{a}(s, s')$ ，以及在下一个状态 $s'$ 时按照策略 $\pi$ 执行所得到的状态价值函数 $V_{\pi}(s')$ 之和，此时也要对状态转移 $p_{a}(s, s')$ 概率求数学期望。

状态价值函数和动作值函数的计算公式称为贝尔曼方程，它们是马尔可夫决策过程的核心。

---

### 马尔可夫决策过程

---

强化学习要解决的问题可以抽象成马尔可夫决策过程（Markov Decision Process，简称MDP）。马尔可夫过程的特点是系统下一个时刻的状态由当前时刻的状态决定，与更早的时刻无关。在MDP中系智能体可以执行动作，从而改变自己和环境的状态，并且得到惩罚或奖励。

**贝尔曼方程**

如果能知道所有状态的状态转移概率，以及回报值，则理论上可以用动态规划算法求解，这是一种迭代法，从一个随机设定的初始值开始，用某种规则进行迭代，直到收敛到状态价值函数或者动作价值函数的极大值。迭代的规则一般是贝尔曼方程或贝尔曼最优性方程。

但在很多实际应用中，无法得到所有状态的转移概率，只能采用随机算法，其核心思想是：  
从一个初始的随机策略开始随机的执行一些动作，然后观察回报和状态转移，以此估计价值函数的值，或者更新价值函数的值。
形象的说，就是加大效果的动作的执行力度，减小效果不好的动作的执行力度。

蒙特卡洛算法和时序差分算法是典型的代表。


## AI & CFD

将物理方程、约束条件等先验知识带入损失函数，通过深度神经网络代理物理数值模型，实现快速模拟、快速响应。

+ 3.1. PINN 方案

---

### PINN 方案

---

+ 3.1. 从无网格方法到内嵌物理知识的神经网络
+ 3.2. 连接数据与物理知识
+ 3.3. 流体力学中的 PINN
+ 3.4. 一个例子

**参考资料：**

+ [PaperWeekly - ​内嵌物理知识神经网络（PINN）是个坑吗？](https://zhuanlan.zhihu.com/p/468748367)

---

3.1. 从无网格方法到内嵌物理知识的神经网络

---

内嵌物理知识神经网络(Physics Informed Neural Network，简称PINN) 是一种科学机器在传统数值领域的应用方法，特别是用于解决与偏微分方程(PDE) 相关的各种问题，包括方程求解、参数反演、模型发现、控制与优化等。

简单概括，PINN 的原理就是通过训练神经网络来最小化损失函数来近似PDE的求解，所谓的损失函数项包括初始和边界条件的残差项，以及区域中选定点（称为"配点"）处的偏微分方程残差。训练完成后进行推断就可以得到时空点上的值了。

PINN 的原型类似无网格方法，比如一种最简单的基于强形式径向基函数的无网格方法 Kansa 法。

这里简单介绍这个方法的原理，考虑这样一个两点边值问题：

$$
f^{''} (x) = -1, x \in (0, 1) \\
f(0) = 0 \\
f(1) = 0
$$

Kansa 法直接假设 $f$ 有以下的形式：

$$
f(x) = \sum_{i = 1}^{N} a_i \phi_i(x)
$$

其中 $\phi_i(x)$ 是基函数，通常会选取某个径向基函数(Radial Basis Function，RBF函数) 的平移。那么在 $(0, 1)$ 区间上选取$N-2$ 个不同的配点 $x_i$ ，这些点并不需要处于特定位置。  
分别得到关于系数 $a_i$  的方程：

$$
f^{''} (x) = \sum_{i = 1}^{N} a_i \phi_{i}^{''}(x) = -1
$$

边值上的两点满足:

$$
f(0) = \sum_{i = 1}^{N} a_i \phi_i(0) = 0 \\
f(1) = \sum_{i = 1}^{N} a_i \phi_i(1) = 0
$$

这样，通过配点和边界一共得到了 $N$ 个关于 $N$  个待定系数 $a_i$ 的线性方程组，通过求解线性组，就可以逼近方程的逼近解了。  
定义 $\phi$ 为RBF，当选取 $\phi_i = \phi(||x - c_i||)$ 时，其实已经用到了一种浅层神经网络，也就是RBF-net。

然而浅层神经网络在学习复杂的特征时可能会力不从心，网络宽度的增加也可能会使线性系统变得非常病态，基函数的超参选取是个问题，配点的选择当然也有讲究。

从数据的角度看，所谓的基就是特征，既然这种线性表征是可行的，那么利用神经网络来进行非线性表征也挺合理。

对于 PINN，对于微分方程：

$$
\mathcal F(u(z);\gamma) = f(z), z \in \Omega \\
\mathcal B(u(z)) = g(z), z \in \partial \Omega \\
\mathcal D(u(z_i)) = d(z_i), i \in D
$$

其中

+ $z$ 是包含了空间和时间的坐标；
+ $u$ 表示方程的解；
+ $\Omega$ 是方程所在的区域；
+ $\mathcal F$ 算子描述了控制方程；
+ $\gamma$ 是控制方程的参数；
+ $\mathcal B$ 算子描述了初值或者边界条件；
+ $\mathcal D$ 算子描述了观测数据的方式；
+ $D$ 是观测数据指标集。

相比传统微分方程数值求解的描述，这里多出了第三行式子，也就是对于数据的使用，这也是PINN的特点之一。当然，PINN模型中这三个条件也不一定全都出现，比如边界条件消失，从传统数值方法的角度来看甚至不能满足定解条件。

然后PINN要做的，就是对解 $u$，用神经网络进行逼近。将这个解用神经网络参数化表达为 $u_\theta$，那么就是要寻找这组参数 $\theta$，使得：

$$
u_\theta(z) \approx u(z), z \in \Omega
$$

通常 $u_\theta$ 具有多层感知机或者加入特殊结构的变种，这里仍然以多层感知机为例：

$$
u_\theta(z) = f_{L} \circ f_{L-1} \circ \dots \circ f_{1}(z)
$$

除最后一层外，其余的各层都是“线性变换+激活函数层”。这里的时空信息都被包含在 $z$ 中，也就可以关于 $z$ 进行自动微分运算来表达 $\mathcal F, \mathcal B, \mathcal D$ 这些微分算子。这只是个最基本的模型，也就是Raissi 2017年底提出的一个网络模型，目前也被使用得最多。

深度学习一些其他网络结构，比如CNN、RNN，通常并没有直接可供直接输入空间或时间 $z$ 的入口，而是将空间或时间的信息直接嵌入到网络本身的结构中。CNN类方法的图像信号天然包含空间信息，RNN类方法的处理单元天然包含时间信息。然后依据“内嵌物理知识”这一思想，  
将微分方程的三个算子 $\mathcal F, \mathcal B, \mathcal D$ **以离散（差分）的方式而不是自动微分的方式嵌入到损失函数中**，  
有时这种内嵌物理的方式会被它们的作者称为是“弱监督”、“自监督”或者“无监督”。  
狭义的PINN并不包含这类不使用自动微分的结构，虽然“内嵌物理”的思想上并没有太大区别。

总之，神经网络 $u_\theta$ 在定义方程三个公式中的残差，就可以引出由三项损失加权得到的总损失：

$$
\mathcal L := \omega_{\mathcal F} \mathcal L_{\mathcal F} + \omega_{\mathcal B} \mathcal L_{\mathcal B} + \omega_{\mathcal D} \mathcal L_{\mathcal D}
$$

这三项其实比较笼统，还可以加入正则化项，以及其他各种先验信息，对于具体问题，细分下来有十多项也正常。最后变成了这样一个优化问题：

$$
\theta^{*} \in arg \min_{\theta} \mathcal L[u_\theta]
$$

注意这跟之前提到的 Kansa 方法在计算上有了重大区别，对于线性微分方程，Kansa 方法通常也会导出线性方程组（矩阵），从而可以使用线性方程求解器对方程进行求解。但PINN这种神经网络不行，即使对于线性方程，也不得不使用非线性求解器（迭代优化器），比如 L-BFGS，或者神经网络训练中用得更多的 SGD、Adam 等。非线性问题的求解通常比线性问题难，这是 PINN 计算效率上一个避不开的障碍。

**总结来说：**

![](./assets/pinn0.png)

+ PINN 是一种（深度）网络，在定义时空区域中给定一个输入点，在训练后在微分方程的该点中产生估计的解。
+ 结合对控制方程的嵌入得到残差，利用残差构造损失项就是 PINN 的一项不太新奇的新奇之处了。本质原理就是将方程（也就是所谓的物理知识）集成到网络中，并使用来自控制方程的残差项来构造损失函数，由该项作为惩罚项来限制可行解的空间。
+ 用 PINN 来求解方程并不需要有标签的数据，比如先前模拟或实验的结果。从这个角度，对 PINN 在深度学习中的地位进行定位的话，大概是处于无监督、自监督、半监督或者弱监督的地位。
+ PINN 算法本质上是一种无网格技术，通过将直接求解控制方程的问题转换为损失函数的优化问题来找到偏微分方程解。

---

3.2. 连接数据与物理知识

---

如果把PINN当作是单纯的数值求解器，通常来讲，不管从速度或者精度，PINN在性能上并不能跟传统方法（有限差分、有限元、有限体积等大类方法）抗衡。那么这种方法能火起来必然有其特色，比如PINN好就好在这种方法或者思想可以弥补科学机器学习领域中单纯数据驱动的弱点。如果把传统数值格式认为是单纯物理知识驱动，那么PINN就是数据驱动与知识驱动的融合方法。

为什么单纯数据驱动会出问题？

首先，纯数据驱动的模型可能非常适合处理大量观测值的应用，但由于外推或观测偏差可能导致泛化性能差，其推断可能并不符合物理，也就是不可信。

另外一个缺点来源于物理现象的混沌本质。

从这个角度，也可以看出“内嵌物理”（physics informed）对于科学机器学习的必要性，作为数据方法与传统知识驱动方程的桥梁存在，也就是“知识”和“数据”共同驱动的方法，如下图所示：

![](./assets/pinn1.png)

对于大多数实际应用，最有趣的部分处于上图中间部分区域，其中用来描述物理场PDE仅有部分信息是已知的，但是可以使用少量的散点测量来推断未知部分的参数（参数恢复）、补全PDE中的某些项（模型发现），同时得到数值解。

---

3.3. 流体力学中的 PINN

---

计算流体力学（Computational Fluid Dynamics，CFD）是各种工程应用中逃不开的学科，说到这个学科，不可避免地会遇到Navier-Stokes（NS）方程（组），或者在一定条件的各类特化数值模型。

对这个方程（或者其简化模型）进行求解已经有了非常多的商业软件，用PINN也能求解，但由于前面提及的PINN高度非线性非凸的本质，直接用于NS方程的求解或许并不是PINN的强项。

CFD与其说是计算科学，更像是一门实验科学，需要物理实验数据来对解算的模型进行验证，还面临诸多困难。比如目前已有的各类CFD方法并不能很好地融合各类保真数据。在工程模型中，可能还涉及**逆问题的求解**，也就是边界条件和流体的各种参数未知的情形下，如何通过部分测量数据得到精确的模型参数和流场的重构。再者，CFD网格质量对结果的影响比较大，计算中网格划分本身也是非常耗时的。最后，目前的CFD软件都非常庞大，比如OpenFoam，对每一类问题都有专门的求解模块，拥有超过10万行的科学计算代码，其更新与维护也是一件难事。

刚好，PINN有部分解决以上问题的能力。PINN对各种数据的融合是非常自然的，不管是压强标量场的时空散点数据、速度矢量场的时空散点数据，还是示踪粒子的运动轨迹，都可以非常容易地融合到PINN的求解中。对于PINN来讲，求解正问题与求解包含数据的逆问题在形式上并没有太大区别，这是它的巨大优势之一。PINN作为一种无网格方法，也不需要网格（当然对应的如何选配点是个别的问题）。最后，PINN的算法核心相对简单，只要NS方程能够描述，都能使用统一的形式进行处理，在更新和维护上也是相对容易的。

从以上的对比就可以发现PINN这种方法的“甜区”了，即在时空散点测量数据比较充足的情况下，进行CFD相关的参数估计、流场重建、代理模型构建等问题的求解。与传统的CFD求解器相比，PINN在集成数据（流量的观测值）和物理知识（其实就是描述该物理现象的控制方程）方面更胜一筹。

基于部分速度测量，下面用一个例子说明PINN用于2维圆柱绕流的未知参数的估计，二维的不可压缩非稳态NS方程为:

$$
u_x + v_y = 0 \\
u_t + \lambda_1 (u u_x + v u_y) = -p_x + \lambda_2(u_{xx} + u_{yy}) \\
v_t + \lambda_1 (u v_x + v v_y) = -p_y + \lambda_2(v_{xx} + v_{yy})
$$

其中 $u(x, y, t)$ 是水平方向上的速度分量，$v(x, y, t)$ 是竖直方向上的速度分量，$p(x, y, t)$ 代表压强场。对流项和黏性项的参数为 $\lambda_1, \lambda_2$，是两个未知量。  
由于不可压缩条件，可以将水平速度和竖直速度写成流函数的偏导数形式，也就是：

$$
u = \psi_y, v= \psi_x
$$

也就是神经网络表示的是 $\psi, p$，替代了 $u, v, p$，通过对 $\psi$ 的偏导数得到两个方向上的速度场。在此条件下，连续性方程天然满足了，不再需要额外的约束。记：

$$
f := u_t + \lambda_1 (u u_x + v u_y) + p_x - \lambda_2(u_{xx} + u_{yy}) \\
g := v_t + \lambda_1 (u v_x + v v_y) + p_y - \lambda_2(v_{xx} + v_{yy})
$$

噪声条件下对速度场的 $N$ 个散点观测值为：

$$
\{t^i, x^i, y^i, u^i, v^i \}_{i=1}^{N}
$$

那么需要极小化下面的误差：

$$
\begin{gathered}\mathrm{MSE}:=\frac{1}{N}\sum_{i=1}^N\left(\left|\psi_y\left(t^i,x^i,y^i\right)-u^i\right|^2+\left|\psi_x\left(t^i,x^i,y^i\right)+v^i\right|^2\right)+\frac{1}{N}\\\sum_{i=1}^N\left(\left|f\left(t^i,x^i,y^i\right)\right|^2+\left|g\left(t^i,x^i,y^i\right)\right|^2\right)\end{gathered}
$$

其中未知变量包括输出 $[\psi(x, y, t), p(x, y, t)]$ 的神经网络参数，以及两个未知的方程参数 $\lambda_1, \lambda_2$。  
圆柱绕流的示意图如下，可以清晰看到卡门涡街现象。

![](./assets/pinn2.webp)

内部的黑框是采样区域，在不同的时间点上采样了N=5000组数据。在没有噪声的情形下，对$\lambda_1, \lambda_2$ 的恢复误差为0.078%与4.67%。

以上是一个比较初步的PINN在流场参数恢复中的作用，是Raissi在17年底文章中的例子。而后，又有了一系列工作。一个比较有前景的应用是流场可视化技术，从观察结果中推断出流场。

比较直接的是通过速度观测来重建全流场。在气动力学等学科的实验研究中，可以利用光学设备，通过粒子图像测速（Particle Image Velocimetry，PIV）和（Particle Tracking Velocimetry，PTV）方法测量的得到多个散点速度。然而散点速度并不能满足需求，高分辨率的速度场对于可视化和后续分析是必不可少的。一个非常自然的想法就是通过类似图像插值来实现从散点到高分辨率流场的“超分辨”，但是这种方式处理得到的结果可能“并不符合物理规律”。

作为融合了物理知识的PINN方法，可以非常自然地从这些稀疏速度信息来重建分辨率的整体速度场。也就是极小化NS方程的损失项，同时得到速度场和压强场，使结果符合“物理规律”。

另一种应用更加神奇，连速度散点测量都没有，直接从流体中载物的浓度来还原流场，其实就是在NS方程组上再添加一个对流扩散方程，代入到PINN进行求解，也就是求解所谓的隐流体力学（Hidden fluid mechanics）：

$$
c_t + u c_x + v c_y + w c_z  = {Pe}^{-1}(c_{xx} + c_{yy} + c_{zz})
$$

其中 $(u, v, w)$ 是流体的三个方向速度，$c$ 是流体中某种载物的浓度。通过测量 $c$ 来恢复出 $(u, v, w)$ ，也就是构造这样一个损失函数：

$$
\begin{aligned}
MSE=\frac1N\sum_{n=1}^{N}\left|c\left(t^{n},x^{n},y^{n},z^{n}\right)-c^{n}\right|^{2} \\
+\sum_{i=1}^5\frac1M\sum_{m=1}^M\left|e_i\left(t^m,x^m,y^m,z^m\right)\right|^2
\end{aligned}
$$

$c^n$ 是观测值，包含了对流扩散方程，对应三个方向的NS方程以及一个不可压缩流体的连续性方程。连初值和边界条件都不需要。这种适合PINN建模求解的逆问题对于传统的CFD求解器来说并不是一件容易的事情，而类似技术被用于颅内血管瘤血流的重建上。

---

3.4. 一个例子

---

考虑下面这样一个方程：

$$
\frac{\partial^2 u}{\partial x^2} - \frac{\partial^4 u}{\partial y^4} = (2 - x^2) e^{-y}\\
u_{yy}(x, 0) = x^2 \\ 
u_{yy}(x, 1) = x^2/e \\
u(x, 0) = x^2 \\
u(x, 1) = x^2/e \\
u(0, y) = 0 \\ 
u(1, y) = e^{-y} 
$$

真解 $u(x, y) = x^2 e^{-y}$。

用PINN来求解方程的话并不需要太多技巧。做法就是定义一个神经网络 $\tilde{u}(x, y;\theta)$，利用神经网络的自动微分机制，可以得到 $\tilde{u}_{xx}, \tilde{u}_{yy}, \tilde{u}_{yyyy}$，然后在区域 $[0, 1] * [0, 1]$ 上进行随机采样和构造损失函数。

上面的方程有一项控制方程和六个边界条件，因此需要构造7部分Loss:

$$
L_1=\frac1{N_1}\sum_{(x_i,y_i)\in\Omega}\left(\tilde{u}_{xx}\left(x_i,y_i;\theta\right)-\tilde{u}_{yyyy}\left(x_i,y_i;\theta\right)-\left(2-x_i^2\right)e^{-y_i}\right)^2 \\
L_2=\frac1{N_2}\sum_{(x_i,y_i)\in[0,1]\times\{0\}}\left(\tilde{u}_{yy}\left(x_i,y_i;\theta\right)-x_i^2\right)^2 \\
L_3=\frac1{N_3}\sum_{(x_i,y_i)\in[0,1]\times\{1\}}\left(\tilde{u}_{yy}\left(x_i,y_i;\theta\right)-\frac{x_i^2}{e} \right)^2 \\
L_4=\frac1{N_4}\sum_{(x_i,y_i)\in[0,1]\times\{0\}}\left(\tilde{u}\left(x_i,y_i;\theta\right)-x_i^2\right)^2 \\
L_5=\frac1{N_5}\sum_{(x_i,y_i)\in[0,1]\times\{1\}}\left(\tilde{u}\left(x_i,y_i;\theta\right)-\frac{x_i^2}{e} \right)^2 \\
L_6=\frac1{N_6}\sum_{(x_i,y_i)\in\{0\}\times[0,1]}\left(\tilde{u}\left(x_i,y_i;\theta\right)-0 \right)^2 \\
L_7=\frac1{N_7}\sum_{(x_i,y_i)\in\{1\}\times[0,1]}\left(\tilde{u}\left(x_i,y_i;\theta\right)-\exp(-y_i)\right)^2
$$

然后就可以构造出总的损失函数：

$$
Loss = \omega_1 L_1 + \omega_2 L_2 + \omega_3 L_3 + \omega_4 L_4 + \omega_5 L_5 + \omega_6 L_6 + \omega_7 L_7
$$


In [ ]:
import torch


# Domain and Sampling
def interior(n=1000):
    x = torch.rand(n, 1)
    y = torch.rand(n, 1)
    cond = (2 - x ** 2) * torch.exp(-y)
    return x.requires_grad_(True), y.requires_grad_(True), cond


def down_yy(n=100):
    x = torch.rand(n, 1)
    y = torch.zeros_like(x)
    cond = x ** 2
    return x.requires_grad_(True), y.requires_grad_(True), cond


def up_yy(n=100):
    x = torch.rand(n, 1)
    y = torch.ones_like(x)
    cond = x ** 2 / torch.e
    return x.requires_grad_(True), y.requires_grad_(True), cond


def down(n=100):
    x = torch.rand(n, 1)
    y = torch.zeros_like(x)
    cond = x ** 2
    return x.requires_grad_(True), y.requires_grad_(True), cond


def up(n=100):
    x = torch.rand(n, 1)
    y = torch.ones_like(x)
    cond = x ** 2 / torch.e
    return x.requires_grad_(True), y.requires_grad_(True), cond


def left(n=100):
    y = torch.rand(n, 1)
    x = torch.zeros_like(y)
    cond = torch.zeros_like(x)
    return x.requires_grad_(True), y.requires_grad_(True), cond


def right(n=100):
    y = torch.rand(n, 1)
    x = torch.ones_like(y)
    cond = torch.exp(-y)
    return x.requires_grad_(True), y.requires_grad_(True), cond


# Neural Network
class MLP(torch.nn.Module):
    def __init__(self):
        super(MLP, self).__init__()
        self.net = torch.nn.Sequential(
            torch.nn.Linear(2, 32),
            torch.nn.Tanh(),
            torch.nn.Linear(32, 32),
            torch.nn.Tanh(),
            torch.nn.Linear(32, 32),
            torch.nn.Tanh(),
            torch.nn.Linear(32, 32),
            torch.nn.Tanh(),
            torch.nn.Linear(32, 1)
        )

    def forward(self, x):
        return self.net(x)


# Loss
loss = torch.nn.MSELoss()


def gradients(u, x, order=1):
    if order == 1:
        return torch.autograd.grad(u, x, grad_outputs=torch.ones_like(u),
                                   create_graph=True,
                                   only_inputs=True, )[0]
    else:
        return gradients(gradients(u, x), x, order=order - 1)


def l_interior(u):
    x, y, cond = interior()
    uxy = u(torch.cat([x, y], dim=1))
    return loss(gradients(uxy, x, 2) - gradients(uxy, y, 4), cond)


def l_down_yy(u):
    x, y, cond = down_yy()
    uxy = u(torch.cat([x, y], dim=1))
    return loss(gradients(uxy, y, 2), cond)


def l_up_yy(u):
    x, y, cond = up_yy()
    uxy = u(torch.cat([x, y], dim=1))
    return loss(gradients(uxy, y, 2), cond)


def l_down(u):
    x, y, cond = down()
    uxy = u(torch.cat([x, y], dim=1))
    return loss(uxy, cond)


def l_up(u):
    x, y, cond = up()
    uxy = u(torch.cat([x, y], dim=1))
    return loss(uxy, cond)


def l_left(u):
    x, y, cond = left()
    uxy = u(torch.cat([x, y], dim=1))
    return loss(uxy, cond)


def l_right(u):
    x, y, cond = right()
    uxy = u(torch.cat([x, y], dim=1))
    return loss(uxy, cond)


# Training

u = MLP()
opt = torch.optim.Adam(params=u.parameters())
for i in range(10000):
    opt.zero_grad()
    l = l_interior(u) \
        + l_up_yy(u) \
        + l_down_yy(u) \
        + l_up(u) \
        + l_down(u) \
        + l_left(u) \
        + l_right(u)
    l.backward()
    opt.step()
    print(i)

# Inference
xc = torch.linspace(0, 1, 100)
xx, yy = torch.meshgrid(xc, xc)
xx = xx.reshape(-1, 1)
yy = yy.reshape(-1, 1)
xy = torch.cat([xx, yy], dim=1)
u_pred = u(xy)
print("Max abs error is: ", float(torch.max(torch.abs(u_pred - xx * xx * torch.exp(-yy)))))